# Análise balanço de empresas

## 1 - Importando as ferramentas

In [1]:
import numpy as np
import pandas as pd
import requests
import string
import warnings
warnings.filterwarnings('ignore')
#pd.options.display.float_format = '{:.6f}'.format


## 2 - Coletando os dados

In [37]:
url = "https://fundamentus.com.br/resultado.php"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

In [38]:
response = requests.get(url, headers=headers)

### 2.1 - Coletando dados formatados

In [39]:
if response.status_code == 200:
    df = pd.read_html(response.text,decimal=',',thousands='.')[0]
else:
    print(f"{response.status_code}")

### 2.2 - Verificando se os dados estão corretos conforme o site

In [40]:
df = df.sort_values("Papel")
display(df.iloc[0])

Papel                       AALR3
Cotação                      10.8
P/L                         -9.98
P/VP                         1.43
PSR                         1.366
Div.Yield                   0,00%
P/Ativo                     0.624
P/Cap.Giro                 -11.34
P/EBIT                      20.91
P/Ativ Circ.Liq             -1.69
EV/EBIT                     28.13
EV/EBITDA                    11.9
Mrg Ebit                    6,53%
Mrg. Líq.                 -12,88%
Liq. Corr.                   0.77
ROIC                        3,32%
ROE                       -14,37%
Liq.2meses               218892.0
Patrim. Líq          1147410000.0
Dív.Brut/ Patrim.            0.64
Cresc. Rec.5a               6,00%
Name: 109, dtype: object

### 2.3 - Ajustando string '%' para número

In [ ]:
df.info()

In [71]:
format_cols = [col for col in df.columns if df[col].dtype == type(object) and col != 'Papel']

def transformToFloat(x):
    return float(str(x).replace('.', '').replace(',', '').rstrip('%')) / 100

In [72]:
for col in format_cols:
    df[col] = df[col].apply(transformToFloat)

### 2. 4 Remover empresas que tenham um liquidez menor que R$1.000.000,00 nos últimos meses

In [75]:
df.shape

(989, 21)

In [78]:
liquidity = 1000000
df = df[df['Liq.2meses'] > liquidity]

In [79]:
df.shape

(221, 21)

### 2.5 Filtrar as empresas que se encaixam na 'Formula Mágica - Joel Greenblatt'

#### 2.5.1 Objetivos
1. Não investir em um título público
2. Investir em uma empresa para participar dos lucros/valorização
3. Só faz sentido investir em uma empresa se o lucro será maior do que um título apesar de todos os riscos

##### Comprar algo valioso quando estiver em um preço de pechincha
1. ROA: Retrono do Capital = EBIT/(Capital de Giro + Ativos Fixos)
2. ROE: Retorno Empresa = EBIT/(Valor de Mercado + Dívida Líq. - Caixa)   
EBIT = Lucro antes de juros ou impostos tirado da DRE (Relatório Contábil de um determinado período - Demonstração do Resultado do Exercício)

In [84]:
ranking = pd.DataFrame()
ranking['pos'] = range(1,151)
# Valor da Firma  = Valor de mercado + Dívida líquida
# Valor de mercado = cotação * nro. de ações
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [85]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,PSSA3
1,2,WIZC3,PLPL3
2,3,SYNE3,ODPV3
3,4,GOAU4,CURY3
4,5,PCAR3,LEVE3
...,...,...,...
145,146,FRAS3,EMBR3
146,147,GOLL4,PFRM3
147,148,PETZ3,RAIZ4
148,149,PORT3,BHIA3


In [86]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [88]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [89]:
t = pd.concat([a,b])

In [91]:
rank = t.dropna(axis=1).sum()

## 3 - Visualizando os dados

In [93]:
rank.sort_values()[:15]

PSSA3      2
WIZC3      8
PLPL3      8
PETR4     25
CMIN3     29
BRBI11    29
PETR3     31
SYNE3     35
KEPL3     38
LEVE3     39
VALE3     43
LREN3     45
VLID3     47
ASAI3     48
SBSP3     51
dtype: int64

In [ ]:
# Filtrar empresas do setor financeiro (sofrem distorções)